In [13]:

# import osrm
import geopandas as gpd
from shapely.geometry import Point, Polygon
import requests
import math
import json

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

# Function to find the center of a polygon
def find_polygon_center(coords):
    lon = sum(coord[0] for coord in coords) / len(coords)
    lat = sum(coord[1] for coord in coords) / len(coords)
    return lat, lon

In [16]:


# Replace with the URL of the OSRM service
OSRM_URL = "http://router.project-osrm.org/route/v1/foot/"

# Function to find the nearest public park (leisure=park) using Overpass API
def find_nearest_public_park(lat, lon):
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
        [out:json];
        (way["leisure"="park"](around:5000,{lat},{lon});
        >;
        );
        out;
    """
    response = requests.get(overpass_url, params={"data": overpass_query})
    try:
        data = response.json()
        if "elements" in data and len(data["elements"]) > 0:
            element = data["elements"][0]
            # Case 2: The API response does not include a "center" field.
            if "type" in element and element["type"] == "way":
                # For "way" type, extract the coordinates from the "geometry" field.
                coords = element["geometry"]["coordinates"]
                
                # Find the center of the polygon
                park_lat, park_lon = find_polygon_center(coords[0])
                park_id = element["id"]
                return park_lat, park_lon, park_id
            else:
                # For other types (e.g., "node" or "relation"), use the latitude and longitude directly.
                park_lat, park_lon, park_id = element["lat"], element["lon"], element["id"]
                return park_lat, park_lon, park_id
    except requests.exceptions.RequestException as e:
        print(f"Error fetching walking directions: {e}")
        return None
    except json.decoder.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        return None


# Function to get walking directions between two points using OSRM
def get_walking_directions(origin, destination):
    url = f"{OSRM_URL}{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception if the response is not successful (status code >= 400)

        # Check if the response is empty
        if not response.text:
            print("Error: Empty response from the server.")
            return None

        # Parse the response as JSON
        data = response.json()

        # Check if the response contains expected data
        if "routes" not in data or not data["routes"]:
            print("Error: Unexpected response data.")
            return None

        # Extract the walking time from the OSRM response
        walking_time = data["routes"][0]["duration"]
        return walking_time

    except requests.exceptions.RequestException as e:
        print(f"Error fetching walking directions: {e}")
        return None
    except json.decoder.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        return None

    
# Function to create a new GeoDataFrame with park locations
def create_park_gdf(polygon_gdf):
    park_rows = []
    for idx, row in polygon_gdf.iterrows():
        polygon = row.geometry
        polygon_center = find_polygon_center(polygon.exterior.coords)
        park_location = find_nearest_public_park(polygon_center[0], polygon_center[1])
        if park_location:
            park_rows.append({"geometry": Point(park_location[1], park_location[0]), "id": park_location[2]})
    if park_rows:
        return gpd.GeoDataFrame(park_rows, crs=polygon_gdf.crs)
    else:
        return None

# Read the origin points from the shapefile using geopandas
def read_origin_points(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    return gdf.geometry.apply(lambda geom: (geom.y, geom.x)).tolist()

# Function to create a new GeoDataFrame with calculated walking times
def create_new_gdf_with_times(origin_gdf, park_gdf):
    new_rows = []
    for idx, row in origin_gdf.iterrows():
        point = row.geometry
        if isinstance(point, Polygon):
            # For Polygon type, use the centroid as the representative point
            point = point.centroid

        nearest_parks = []
        for _, park_row in park_gdf.iterrows():
            park_point = park_row.geometry
            distance = haversine_distance(point.y, point.x, park_point.y, park_point.x)
            nearest_parks.append((park_point.y, park_point.x, distance, park_row["id"]))
        nearest_parks.sort(key=lambda x: x[2])  # Sort by distance (third element of the tuple)
        nearest_park_info = nearest_parks[0]  # Select the closest park
        park_lat, park_lon, distance, park_id = nearest_park_info
        walking_time = get_walking_directions((point.y, point.x), (park_lat, park_lon))
        if walking_time:
            new_rows.append({"geometry": point, "walking_time": walking_time, "park_id": park_id})
    if new_rows:
        return gpd.GeoDataFrame(new_rows, crs=origin_gdf.crs)
    else:
        return None
    
# Example shapefile path for the grid of polygons
polygon_shapefile_path = "C:\_DEV\CODE\DIVERS\python\_Park&walk\carroyage_filosofi_insee_2015.shp"

# Create a new GeoDataFrame with park locations
polygon_gdf = gpd.read_file(polygon_shapefile_path)
park_gdf = create_park_gdf(polygon_gdf)

# Function to create a new GeoDataFrame with calculated walking times
result_gdf = create_new_gdf_with_times(polygon_gdf, park_gdf)


###########################
######## EXPORTS #########
###########################


# Save the result GeoDataFrame to a new shapefile
if result_gdf is not None:
    result_shapefile_path = "C:\_DEV\CODE\DIVERS\python\_Park&walk\points_walk_results.shp"
    result_gdf.to_file(result_shapefile_path)
    print("Result saved as a new shapefile.")
else:
    print("No walking times calculated or no public parks found.")

# Save the park GeoDataFrame to a new shapefile
if park_gdf is not None:
    park_shapefile_path = "C:\_DEV\CODE\DIVERS\python\_Park&walk\public_parks.shp"
    park_gdf.to_file(park_shapefile_path)
    print("Park locations saved as a new shapefile.")
else:
    print("No public parks found.")


        [out:json];
        (way["leisure"="park"](around:1000,43.604957241873,1.4129223866235339);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.56334573741974,1.4141207402531493);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.60516341781826,1.415381238633722);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.605369534981406,1.417840106231353);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.602892249534676,1.3883347247583657);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.601236023795856,1.3686657206979507);
        >;
        );
        out;
    

        [out:json];
        (way["leisure"="park"](around:1000,43.60475100714619,1.410463550205002);
        >;
        );
        out;
    



KeyboardInterrupt: 

Result saved as a new shapefile.
Park locations saved as a new shapefile.


C:\Users\tlecae\AppData\Local\Temp\ipykernel_16796\2471922267.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result_gdf.to_file(result_shapefile_path)
